<a href="https://colab.research.google.com/github/peterbmob/CH-PFC/blob/main/spectral_solve_fen3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%
# ch_spectral_fen.py  (masked + BV kinetics)
# -------------------------------------------------------------
# Spectral CH solver aligned with the FEniCS nondimensionalization
# and elasticity from input 3.py, augmented with a smoothed-
# boundary mask H for particle-in-reservoir and a Butler–Volmer
# surface flux converted to a volumetric source via δΓ ≈ |∇H|.
# -------------------------------------------------------------
from __future__ import annotations
import numpy as np
from numpy.fft import fftn, ifftn, fftfreq
import importlib, sys, os



# ---------- Load config (accepts config.py or 'config 3.py') ----------
def _try_import_config():
    try:
        return importlib.import_module('config')
    except Exception:
        cfg_path = os.path.join(os.getcwd(), 'config 3.py')
        if os.path.exists(cfg_path):
            spec = importlib.util.spec_from_file_location('config_3', cfg_path)
            mod = importlib.util.module_from_spec(spec)
            sys.modules['config_3'] = mod
            assert spec.loader is not None
            spec.loader.exec_module(mod)
            return mod
        raise

_config = _try_import_config()
Adapt   = getattr(_config, 'Adapt')
Domain  = getattr(_config, 'Domain')
Interval= getattr(_config, 'Interval')
Model   = getattr(_config, 'Model')

# ---------- FE-aligned nondimensionalization ----------
Wc    = float(Model["Wc"])               # [m]
sigma = float(Model["sigma"])            # [J/m^2]
DLi   = float(Model["DLi"])              # [m^2/s]
R     = float(Model["R"])                # [J/mol/K]
To    = float(Model["To"])               # [K]
vm    = float(Model["vm"])               # [m^3/mol]
Omega = float(Model["Omega"])            # [J/mol]
E     = float(Model["E"])                # [Pa]
nu    = Model.get('nu', Model.get('ν'))
if nu is None:
    raise KeyError("Poisson ratio 'nu'/'ν' not found in Model")
nu    = float(nu)

# Electrochem params for BV (mirror FE)
Fconst = float(Model.get('F', 96485.33))
NA     = float(Model.get('Nₐ', Model.get('Nα', Model.get('NA', 6.02214076e23))))
DeltaPhi = float(Model.get('Δφ', Model.get('DeltaPhi', 0.0)))  # [V]
mu_eq    = float(Model.get('μeq', Model.get('mueq', 0.0)))     # [J/m^3]
alpha    = 0.5  # transfer coefficient

# j0 or k0 -> FE’s dimensionless j0coeff
j0 = Model.get('j0', None)
if j0 is None:
    k0 = float(Model.get('k0', 2.035e-4))  # [s^-1]
    j0 = (Fconst/(NA * Wc**2)) * k0        # [A/m^2]
else:
    j0 = float(j0)

Hscale = sigma / Wc                 # H = sigma/Wc  [J/m^3]
tc     = Wc**2 / DLi                # [s]
RTv    = (R * To / vm) / Hscale     # dimensionless
Om     = (Omega / vm) / Hscale      # dimensionless
Dm     = 1.0 / RTv                  # dimensionless mobility prefactor
j0coeff = (vm * j0 / Fconst) * (tc / Wc)  # dimensionless (exactly like FE)
mu_elec = (mu_eq - (Fconst * DeltaPhi)/vm) / Hscale  # dimensionless μ_elec

# ---------- Domain & grid (lengths in Wc units) ----------
Lx = float(Domain["Lx"]); Ly = float(Domain["Ly"])
nde = float(Domain["nde"])
Nx = max(8, int(round(Lx/nde))); Ny = max(8, int(round(Ly/nde)))
if Nx % 2: Nx += 1
if Ny % 2: Ny += 1
x = np.linspace(0.0, Lx, Nx, endpoint=False)
y = np.linspace(0.0, Ly, Ny, endpoint=False)
X, Y = np.meshgrid(x, y, indexing='ij')

kx = 2*np.pi*fftfreq(Nx, d=Lx/Nx)
ky = 2*np.pi*fftfreq(Ny, d=Ly/Ny)
KX, KY = np.meshgrid(kx, ky, indexing='ij')
K2 = KX**2 + KY**2
K2[0,0] = 1e-30


# Grid spacings (dimensionless lengths, in units of Wc)
dx = Lx / Nx
dy = Ly / Ny

# Optional out-of-plane thickness (meters) for 2D -> 3D current [A]
# If not provided, use Wc (one diffuse-interface length) as a physically
# meaningful thickness scale. You can set Model['depth'] to anything you like.
depth = float(Model.get('depth', Wc))  # [m]

# Convert from dimensionless BV flux J to *physical* current [A]
# j_phys [A/m^2] = (F/vm) * (Wc/tc) * J_dimless
# For the smoothed boundary, total current:
# I [A] = ∫ j_phys |∇H|_phys dA_phys * depth
# Using |∇H|_phys = |∇H|/Wc and dA_phys = (dx*Wc)*(dy*Wc),
# I = (F/vm) * (Wc^2/tc) * depth * ∑ J |∇H| dx dy
Iconv = (Fconst / vm) * (Wc**2 / tc) * depth  # multiply by sum(J*delta_Gamma)*dx*dy

# ---------- Mask H(X) + smoothed boundary δΓ ----------
# shape: 'diamond' | 'hex' | 'circle' | 'slab_x' | 'slab_y'
mask_shape    = 'diamond' # Default
r_electrode_Wc= 0.39*min(Lx,Ly)
center = (Lx/2, Ly/2)
sigma_smooth = min(Lx/Nx, Ly/Ny)  # ~1 grid cell

# Parameters for 'slab_x' shape
slab_x_center = Lx / 2.0
slab_x_width  = Lx / 4.0 # Example width

# Parameters for 'slab_y' shape (New)
slab_y_center = Ly / 2.0
slab_y_width  = Ly / 4.0 # Example width


import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 130

def wavg(field: np.ndarray, weight: np.ndarray) -> float:
    """Weighted average with nonzero safeguard."""
    wsum = np.sum(weight)
    return float(np.sum(weight * field) / max(wsum, 1e-30))

def make_mask(shape='diamond', r=r_electrode_Wc, center=center,
              slab_x_center=slab_x_center, slab_x_width=slab_x_width,
              slab_y_center=slab_y_center, slab_y_width=slab_y_width): # Added slab_y params
    cx, cy = center
    XX = (X - cx); YY = (Y - cy)
    if shape == 'diamond':
        raw = (np.abs(XX) + np.abs(YY)) <= r
    elif shape == 'hex':
        s3o2 = np.sqrt(3)/2
        ax, ay = np.abs(XX), np.abs(YY)
        raw = np.maximum(ay, s3o2*ax + 0.5*ay) <= r
    elif shape == 'slab_x':
        raw = np.abs(X - slab_x_center) <= slab_x_width / 2.0
    elif shape == 'slab_y': # Added slab_y shape
        raw = np.abs(Y - slab_y_center) <= slab_y_width / 2.0
    else:  # circle
        raw = (XX**2 + YY**2) <= r**2
    return raw.astype(float)

def gaussian_filter_field(f, sigma_len):
    G = np.exp(-0.5*((KX*sigma_len)**2 + (KY*sigma_len)**2))
    return np.real(ifftn(fftn(f)*G))

def grad_field(f):
    fk = fftn(f)
    fx = np.real(ifftn(1j*KX*fk))
    fy = np.real(ifftn(1j*KY*fk))
    return fx, fy

# H_raw and H are calculated in the demo run cell (C_4r2LbeF4nc) based on mask_shape
# H_raw = make_mask(mask_shape, r_electrode_Wc, center, slab_x_center, slab_x_width)
# H = gaussian_filter_field(H_raw, sigma_smooth)
# Hx, Hy = grad_field(H)
# delta_Gamma = np.sqrt(Hx**2 + Hy**2) + 1e-14

# ---------- Elasticity (dimensionless) ----------
lam = E*nu/((1+nu)*(1-2*nu)); mu  = E/(2*(1+nu))
lam_d = lam / Hscale;          mu_d  = mu  / Hscale
C = np.zeros((2,2,2,2))
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                C[i,j,k,l] = lam_d*(1 if i==j else 0)*(1 if k==l else 0) \
                             + mu_d*((1 if i==k else 0)*(1 if j==l else 0)
                                    + (1 if i==l else 0)*(1 if j==k else 0))

e11 = float(Model.get('e11', 0.0)); e22 = float(Model.get('e22', 0.0))
Eps0 = np.zeros((2,2)); Eps0[0,0]=e11; Eps0[1,1]=e22

K = np.stack((KX, KY), axis=-1)
A = np.einsum('...j,ijkl,...k->...il', K, C, K)
A11=A[...,0,0]; A12=A[...,0,1]; A21=A[...,1,0]; A22=A[...,1,1]
detA = A11*A22 - A12*A21
mask0 = (np.abs(KX)<1e-14) & (np.abs(KY)<1e-14)
detA[mask0]=1.0
invA = np.empty_like(A)
invA[...,0,0]=A22/detA; invA[...,0,1]=-A12/detA
invA[...,1,0]=-A21/detA; invA[...,1,1]=A11/detA

# ---------- Free energy (dimensionless) ----------
eps_clip = 1e-12
def f_chem(c):
    ce = np.clip(c, eps_clip, 1.0-eps_clip)
    return RTv*(ce*np.log(ce) + (1-ce)*np.log(1-ce)) + Om*ce*(1.0-ce)

def dfdc_chem(c):
    ce = np.clip(c, eps_clip, 1.0-eps_clip)
    return RTv*(np.log(ce) - np.log(1.0-ce)) + Om*(1.0 - 2.0*ce)

def laplace(f):
    return np.real(ifftn(-K2*fftn(f)))

# ---------- Elastic solver ----------
def solve_elastic(c):
    E0 = np.zeros((Nx,Ny,2,2))
    E0[...,0,0] = c*Eps0[0,0]; E0[...,1,1] = c*Eps0[1,1]
    E0k = np.zeros_like(E0, dtype=complex)
    for a in range(2):
        for b in range(2):
            E0k[...,a,b] = fftn(E0[...,a,b])
    b = 1j*np.einsum('...j,ijkl,...kl->...i', K, C, E0k)
    u_k = np.einsum('...ij,...j->...i', invA, b)
    u_k[mask0,...]=0.0
    Ux=u_k[...,0]; Uy=u_k[...,1]
    Exx = np.real(ifftn(1j*KX*Ux))
    Eyy = np.real(ifftn(1j*KY*Uy))
    Exy = np.real(ifftn(0.5j*(KX*Uy + KY*Ux)))
    DE = np.zeros_like(E0)
    DE[...,0,0]=Exx - E0[...,0,0]
    DE[...,1,1]=Eyy - E0[...,1,1]
    DE[...,0,1]=Exy; DE[...,1,0]=Exy
    sigma = np.einsum('ijkl,...kl->...ij', C, DE)
    f_el = 0.5*np.einsum('...ij,ijkl,...kl->...', DE, C, DE)
    mu_el = -(sigma[...,0,0]*Eps0[0,0] + sigma[...,1,1]*Eps0[1,1] + 2.0*sigma[...,0,1]*Eps0[0,1])
    return mu_el, f_el

# ---------- BV kinetics (dimensionless, FE-like) ----------
def J_BV(mu):
    eta = (mu_elec - mu) / RTv
    return j0coeff * (np.exp(alpha*eta) - np.exp(-(1.0-alpha)*eta))

# ---------- CH step ----------
Mlin = Dm*0.25
Ak = 1.0 + float(Interval['timestep'])*Mlin*(K2**2)

def divergence_of_M_grad_mu(Mc, mu):
    muk = fftn(mu)
    mux = np.real(ifftn(1j*KX*muk))
    muy = np.real(ifftn(1j*KY*muk))
    jx = Mc*mux
    jy = Mc*muy
    return np.real(ifftn(1j*KX*fftn(jx) + 1j*KY*fftn(jy)))

def step_CH(c, dt):
    mu_el, f_el = solve_elastic(c)
    mu = dfdc_chem(c) - laplace(c) + mu_el
    Mc = Dm * (H * c * (1.0 - c))               # diffusion only in particle
    div_term = divergence_of_M_grad_mu(Mc, mu)
    J = J_BV(mu)                                 # boundary flux (dimless)
    Rsrc = J * delta_Gamma                       # smoothed-boundary source
    rhs = c + dt*(div_term + Rsrc)
    c_new = np.real(ifftn(fftn(rhs)/Ak))
    c_new = H*c_new + (1.0 - H)*1.0              # reservoir outside
    return np.clip(c_new, 1e-8, 1.0-1e-8), {'mu':mu, 'mu_el':mu_el, 'f_el':f_el, 'J':J}

In [2]:
import matplotlib
matplotlib.use("Agg")  # safe for headless runs
import matplotlib.pyplot as plt
import os

# ---- Movie/frame settings ----
save_frames  = True
frame_every  = 50          # save a frame every 50 steps
frame_dir    = "frames"    # directory to write PNG frames
cmap_name    = "viridis"   # colormap

def save_concentration_frame(it, t_dimless, c, H, Lx, Ly):
    """Save a concentration frame PNG with the H=0.5 red boundary."""
    if not os.path.isdir(frame_dir):
        os.makedirs(frame_dir, exist_ok=True)
    extent = [0, Lx, 0, Ly]
    plt.figure(figsize=(5.2, 4.2))
    im = plt.imshow(c.T, origin='lower', extent=extent, vmin=0, vmax=1, cmap=cmap_name)
    plt.contour(H.T, levels=[0.5], colors='red', linewidths=0.8, origin='lower', extent=extent)
    plt.colorbar(im, fraction=0.046)
    plt.title(f"c, step {it}  (t = {t_dimless*tc:.3e} s)")
    plt.xlabel("x [Wc]"); plt.ylabel("y [Wc]")
    plt.tight_layout()
    fname = os.path.join(frame_dir, f"frame_{it:06d}.png")
    plt.savefig(fname, dpi=150)
    plt.close()


In [ ]:
# ---------- Demo run ----------
dt = float(Interval['timestep'])
nsteps = 20000 #min(5000, int(Interval.get('maxtimestep', 1000)))


print("--- ch_spectral_fen.py (masked + BV) ---")
print(f"Grid: {Nx}x{Ny}, L=({Lx},{Ly}) Wc; dt={dt:.3e}; RTv={RTv:.3e}, Om={Om:.3e}, Dm={Dm:.3e}")
print(f"BV: j0coeff={j0coeff:.3e}, mu_elec={mu_elec:.3e} (dimless), alpha={alpha}")


# --- logging arrays ---
times_s = []        # time in seconds
currents_A = []     # total current in Amperes
c_part_hist = []    # H-weighted mean concentration (particle)
c_dom_hist  = []    # domain mean concentration
V_hist = []         # voltage proxy in Volts

t_dimless = 0.0

# Reference voltage (optional, for plotting) – adjust if desired
V_ref = float(Model.get('V_ref', 3.45))  # V vs Li/Li+ (typical LiFePO4)

t_dimless = 0.0  # dimensionless time

# Set mask shape to 'slab_y'
mask_shape='slab_y' # Corrected typo from maske_shape

# Recalculate H, Hx, Hy, delta_Gamma based on mask_shape
# This block needs to run BEFORE the initial condition is set
H_raw = make_mask(mask_shape, r_electrode_Wc, center, slab_x_center, slab_x_width, slab_y_center, slab_y_width)
H = gaussian_filter_field(H_raw, sigma_smooth)
Hx, Hy = grad_field(H)
delta_Gamma = np.sqrt(Hx**2 + Hy**2) + 1e-14


# Initial condition: smoothed interface with different concentrations inside/outside H_raw
c_inside = 0.10  # Concentration inside the particle (where H_raw is ~1)
c_outside = 1.0  # Concentration outside the particle (where H_raw is ~0)

# Use the smoothed mask H to create the initial condition
# This creates a smooth transition between c_inside and c_outside
c = c_inside * H + c_outside * (1.0 - H)


rng = np.random.default_rng(0)
# Add small noise everywhere, or just near the boundary?
# Adding noise everywhere for simplicity, might refine later if needed.
c = c + (0.01 * (rng.random((Nx,Ny)) - 0.5))

c = np.clip(c, 1e-3, 1.0-1e-3)


for it in range(1, nsteps+1):
    c, info = step_CH(c, dt)

    # Advance dimensionless time and log physical time
    t_dimless += dt

    # Save a frame every 'frame_every' steps
    if save_frames and (it % frame_every == 0 or it == 1):
        save_concentration_frame(it, t_dimless, c, H, Lx, Ly)

    times_s.append(t_dimless * tc)

    # Total current (physical, A) via smoothed boundary integral
    J = info['J']  # dimensionless BV flux
    Ssum = np.sum(J * delta_Gamma) * dx * dy    # dimensionless integral
    I_A = Iconv * Ssum                           # [A]
    currents_A.append(I_A)

    # Mean concentrations
    c_part = wavg(c, H)      # H-weighted (particle average)
    c_dom  = float(c.mean()) # whole domain
    c_part_hist.append(c_part)
    c_dom_hist.append(c_dom)

    # Voltage proxy (Volts): V ≈ V_ref + ( <mu> * Hscale * vm / F ) - Δφ
    # Use particle-weighted average of mu
    mu_avg_dimless = wavg(info['mu'], H)
    V_proxy = V_ref + (mu_avg_dimless * Hscale * vm / Fconst) - DeltaPhi
    V_hist.append(V_proxy)

    if it % 50 == 0 or it == 1:
        fchem = f_chem(c).mean(); fel=info['f_el'].mean(); Jm=info['J']*delta_Gamma
        print(f"step {it:4d} | <c>={c.mean():.4f} <f_chem>={fchem:.3e} <f_el>={fel:.3e} I≈{(Jm.sum()*(Lx/Nx)*(Ly/Ny)):.3e}")

np.savez('ch_spectral_fen_checkpoint.npz',
          c=c, Nx=Nx, Ny=Ny, Lx=Lx, Ly=Ly,
          RTv=RTv, Om=Om, Dm=Dm, lam_d=lam_d, mu_d=mu_d,
          H=H,                       # <-- add this
          delta_Gamma=delta_Gamma    # <-- optional but handy
)
print('Saved: ch_spectral_fen_checkpoint.npz')

--- ch_spectral_fen.py (masked + BV) ---
Grid: 512x256, L=(128.0,64.0) Wc; dt=1.000e-03; RTv=7.857e-01, Om=3.805e+00, Dm=1.273e+00
BV: j0coeff=1.480e-02, mu_elec=-2.931e-13 (dimless), alpha=0.5
step    1 | <c>=0.7754 <f_chem>=2.615e-02 <f_el>=2.935e-01 I≈-2.516e+00
step   50 | <c>=0.7904 <f_chem>=2.580e-02 <f_el>=2.993e-01 I≈-8.735e+02
step  100 | <c>=0.7939 <f_chem>=2.601e-02 <f_el>=3.000e-01 I≈-8.529e+02
step  150 | <c>=0.7961 <f_chem>=2.611e-02 <f_el>=3.005e-01 I≈-7.932e+02
step  200 | <c>=0.7977 <f_chem>=2.614e-02 <f_el>=3.008e-01 I≈-7.781e+02
step  250 | <c>=0.7989 <f_chem>=2.612e-02 <f_el>=3.011e-01 I≈-1.196e+03
step  300 | <c>=0.8000 <f_chem>=2.610e-02 <f_el>=3.014e-01 I≈-8.306e+02
step  350 | <c>=0.8009 <f_chem>=2.605e-02 <f_el>=3.016e-01 I≈-5.557e+02
step  400 | <c>=0.8016 <f_chem>=2.598e-02 <f_el>=3.018e-01 I≈-8.807e+02
step  450 | <c>=0.8023 <f_chem>=2.590e-02 <f_el>=3.020e-01 I≈-1.475e+03
step  500 | <c>=0.8029 <f_chem>=2.586e-02 <f_el>=3.021e-01 I≈-4.974e+02
step  550 | <c

In [ ]:
#!/usr/bin/env python3
"""
Analyze a spectral CH checkpoint saved by ch_spectral_fen.py
and produce diagnostic plots for fields and energies.

Usage:
  python analyze_checkpoint.py [checkpoint.npz]

If no file is given, defaults to 'ch_spectral_fen_checkpoint.npz'.
"""
from __future__ import annotations
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fftn, ifftn, fftfreq
import sys, os, importlib
import importlib.util # Moved this import to the top

# --- Load checkpoint ---
# Adjusting to handle potential extra arguments in notebook environments
fn = 'ch_spectral_fen_checkpoint.npz'
# Filter out potential non-filename arguments, being more specific about expected file extensions
potential_fns = [arg for arg in sys.argv[1:] if arg.endswith('.npz')]
if potential_fns:
    fn = potential_fns[0]

data = np.load(fn, allow_pickle=True)

c  = data['c']               # (Nx,Ny)
Nx = int(data['Nx']); Ny = int(data['Ny'])
Lx = float(data['Lx']); Ly = float(data['Ly'])
RTv = float(data['RTv']); Om = float(data['Om'])
Dm  = float(data['Dm'])
lam_d = float(data['lam_d']); mu_d = float(data['mu_d'])
H = data['H']  # shape (Nx, Ny)
extent = [0, Lx, 0, Ly]
# --- Import config to get eigenstrains (e11,e22) ---
# Accept either config.py or 'config 3.py' as module.

def _try_import_config():
    try:
        return importlib.import_module('config')
    except Exception:
        cfg_path = os.path.join(os.getcwd(), 'config 3.py')
        if os.path.exists(cfg_path):
            spec = importlib.util.spec_from_file_location('config_3', cfg_path)
            mod = importlib.util.module_from_spec(spec)
            sys.modules['config_3'] = mod
            assert spec.loader is not None
            spec.loader.exec_module(mod)
            return mod
        raise

_config = _try_import_config()
Model = getattr(_config, 'Model')
e11 = float(Model.get('e11', 0.0))
e22 = float(Model.get('e22', 0.0))

# --- Grid and wavenumbers ---
x = np.linspace(0.0, Lx, Nx, endpoint=False)
y = np.linspace(0.0, Ly, Ny, endpoint=False)
KX = 2*np.pi*fftfreq(Nx, d=Lx/Nx)
KY = 2*np.pi*fftfreq(Ny, d=Ly/Ny)
KX, KY = np.meshgrid(KX, KY, indexing='ij')
K2 = KX**2 + KY**2
K2[0,0] = 1e-30

# --- Free-energy pieces (dimensionless) ---
eps = 1e-12

def f_chem(c):
    ce = np.clip(c, eps, 1.0-eps)
    return RTv * (ce*np.log(ce) + (1-ce)*np.log(1-ce)) + Om * ce*(1.0-ce)

def dfdc_chem(c):
    ce = np.clip(c, eps, 1.0-eps)
    return RTv*(np.log(ce) - np.log(1.0-ce)) + Om*(1.0-2.0*ce)

def laplace(f):
    return np.real(ifftn(-K2*fftn(f)))

# --- Elasticity ---
# Build 4th-rank isotropic stiffness (dimensionless)
C = np.zeros((2,2,2,2))
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                C[i,j,k,l] = lam_d*(1.0 if i==j else 0.0)*(1.0 if k==l else 0.0) \
                             + mu_d*((1.0 if i==k else 0.0)*(1.0 if j==l else 0.0) \
                                    + (1.0 if i==l else 0.0)*(1.0 if j==k else 0.0))
Eps0 = np.zeros((2,2)); Eps0[0,0]=e11; Eps0[1,1]=e22

K = np.stack((KX, KY), axis=-1)
A = np.einsum('...j,ijkl,...k->...il', K, C, K)
A11=A[...,0,0]; A12=A[...,0,1]; A21=A[...,1,0]; A22=A[...,1,1]
detA = A11*A22 - A12*A21
mask0 = (np.abs(KX)<1e-14) & (np.abs(KY)<1e-14)
detA[mask0]=1.0
invA = np.empty_like(A)
invA[...,0,0]=A22/detA; invA[...,0,1]=-A12/detA
invA[...,1,0]=-A21/detA; invA[...,1,1]=A11/detA

# Solve elasticity for given c
E0 = np.zeros((Nx,Ny,2,2))
E0[...,0,0]=c*Eps0[0,0]
E0[...,1,1]=c*Eps0[1,1]
E0k = np.zeros_like(E0, dtype=complex)
for a in range(2):
    for b in range(2):
        E0k[...,a,b] = fftn(E0[...,a,b])

b = 1j*np.einsum('...j,ijkl,...kl->...i', K, C, E0k)
u_k = np.einsum('...ij,...j->...i', invA, b)
u_k[mask0,...]=0.0
Ux = u_k[...,0]; Uy=u_k[...,1]
Exx_k = 1j*KX*Ux; Eyy_k = 1j*KY*Uy; Exy_k = 0.5j*(KX*Uy + KY*Ux)
Exx = np.real(ifftn(Exx_k)); Eyy=np.real(ifftn(Eyy_k)); Exy=np.real(ifftn(Exy_k))
# dE = eps - E0
E0xx=E0[...,0,0]; E0yy=E0[...,1,1]
dExx = Exx - E0xx; dEyy = Eyy - E0yy; dExy = Exy
# Stress sigma = C : dE
sigma = np.zeros((Nx,Ny,2,2))
# Build dE tensor field to contract
DE = np.zeros_like(E0)
DE[...,0,0]=dExx; DE[...,1,1]=dEyy; DE[...,0,1]=dExy; DE[...,1,0]=dExy
sigma = np.einsum('ijkl,...kl->...ij', C, DE)

# Energies and potentials
f_el = 0.5*np.einsum('...ij,ijkl,...kl->...', DE, C, DE)
mu_el = -(sigma[...,0,0]*Eps0[0,0] + sigma[...,1,1]*Eps0[1,1] + 2.0*sigma[...,0,1]*Eps0[0,1])
mu = dfdc_chem(c) - laplace(c) + mu_el

# Gradient energy density 1/2 |∇c|^2
ck = fftn(c)
dcx = np.real(ifftn(1j*KX*ck)); dcy = np.real(ifftn(1j*KY*ck))
f_grad = 0.5*(dcx**2 + dcy**2)

# --- Print & plot ---
print("=== Analysis of checkpoint ===")
print(f"<c> = {c.mean():.6f}, std(c) = {c.std():.6f}")
print(f"<f_chem> = {f_chem(c).mean():.6e}")
print(f"<f_grad> = {f_grad.mean():.6e}")
print(f"<f_el>   = {f_el.mean():.6e}")
print(f"<F_total>= {(f_chem(c)+f_grad+f_el).mean():.6e}")

extent=[0,Lx,0,Ly]
fig,axs=plt.subplots(2,3,figsize=(11,7))
im=axs[0,0].imshow(c.T,origin='lower',extent=extent); axs[0,0].set_title('c')
plt.colorbar(im,ax=axs[0,0])

# <-- Add thin red boundary line at H=0.5:
axs[0,0].contour(
    H.T,
    levels=[0.5],
    colors='red',
    linewidths=0.8,
    extent=extent # thin line
)
im=axs[0,1].imshow(mu.T,origin='lower',extent=extent); axs[0,1].set_title('mu')
plt.colorbar(im,ax=axs[0,1])
im=axs[0,2].imshow(mu_el.T,origin='lower',extent=extent); axs[0,2].set_title('mu_el')
plt.colorbar(im,ax=axs[0,2])
im=axs[1,0].imshow(f_el.T,origin='lower',extent=extent); axs[1,0].set_title('f_el')
plt.colorbar(im,ax=axs[1,0])
im=axs[1,1].imshow(f_chem(c).T,origin='lower',extent=extent); axs[1,1].set_title('f_chem')
plt.colorbar(im,ax=axs[1,1])
im=axs[1,2].imshow(f_grad.T,origin='lower',extent=extent); axs[1,2].set_title('f_grad')
plt.colorbar(im,ax=axs[1,2])
plt.tight_layout()
plt.show()
plt.savefig('analysis_fields.png',dpi=150)
print('Saved: analysis_fields.png')

In [ ]:
#!/usr/bin/env python3
"""
Sweep uniform concentration c and compute chemical & elastic energy
under the same normalization as ch_spectral_fen.py (periodic BCs).

Usage:
  python analyze_sweep_uniform.py [Nx Ny Lx Ly]
Defaults: read from checkpoint if present; otherwise (256,128,64,32).
"""
from __future__ import annotations
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fftn, ifftn, fftfreq
import os, importlib, sys
import importlib.util # Moved this import to the top

# Try to read checkpoint for grid/scales
Nx=256; Ny=128; Lx=64.0; Ly=32.0
RTv=0.7857; Om=3.805; lam_d=708.5; mu_d=697.2
try:
    data=np.load('ch_spectral_fen_checkpoint.npz', allow_pickle=True)
    Nx=int(data['Nx']); Ny=int(data['Ny']); Lx=float(data['Lx']); Ly=float(data['Ly'])
    RTv=float(data['RTv']); Om=float(data['Om'])
    lam_d=float(data['lam_d']); mu_d=float(data['mu_d'])
except Exception:
    pass

# Allow cmdline overrides
if len(sys.argv)>=5:
    Nx=int(sys.argv[1]); Ny=int(sys.argv[2]); Lx=float(sys.argv[3]); Ly=float(sys.argv[4])

# Import config for eigenstrain

def _try_import_config():
    try:
        return importlib.import_module('config')
    except Exception:
        cfg_path = os.path.join(os.getcwd(), 'config 3.py')
        if os.path.exists(cfg_path):
            spec = importlib.util.spec_from_file_location('config_3', cfg_path)
            mod = importlib.util.module_from_spec(spec)
            sys.modules['config_3'] = mod
            assert spec.loader is not None
            spec.loader.exec_module(mod)
            return mod
        raise

_config=_try_import_config()
Model=getattr(_config,'Model')
e11=float(Model.get('e11',0.0)); e22=float(Model.get('e22',0.0))

# Grid and operators
KX = 2*np.pi*fftfreq(Nx, d=Lx/Nx); KY=2*np.pi*fftfreq(Ny, d=Ly/Ny)
KX,KY=np.meshgrid(KX,KY,indexing='ij')
K2=KX**2+KY**2; K2[0,0]=1e-30

# Stiffness tensor
C=np.zeros((2,2,2,2))
for i in range(2):
  for j in range(2):
    for k in range(2):
      for l in range(2):
        C[i,j,k,l]=lam_d*(1 if i==j else 0)*(1 if k==l else 0) + mu_d*((1 if i==k else 0)*(1 if j==l else 0)+(1 if i==l else 0)*(1 if j==k else 0))
Eps0=np.zeros((2,2)); Eps0[0,0]=e11; Eps0[1,1]=e22

K=np.stack((KX,KY),axis=-1)
A=np.einsum('...j,ijkl,...k->...il',K,C,K)
A11=A[...,0,0];A12=A[...,0,1];A21=A[...,1,0];A22=A[...,1,1]
detA=A11*A22-A12*A21
mask0=(np.abs(KX)<1e-14)&(np.abs(KY)<1e-14)
detA[mask0]=1.0
invA=np.empty_like(A)
invA[...,0,0]=A22/detA; invA[...,0,1]=-A12/detA
invA[...,1,0]=-A21/detA; invA[...,1,1]=A11/detA

# helpers

def f_chem(c):
    ce=np.clip(c,1e-12,1-1e-12)
    return RTv*(ce*np.log(ce)+(1-ce)*np.log(1-ce))+Om*ce*(1-ce)

# sweep
cs=np.linspace(0.01,0.99,120)
fel=[]; fchem=[]; ftotal=[]
for ci in cs:
    c=ci*np.ones((Nx,Ny))
    # eigenstrain
    E0=np.zeros((Nx,Ny,2,2)); E0[...,0,0]=c*Eps0[0,0]; E0[...,1,1]=c*Eps0[1,1]
    E0k=np.zeros_like(E0,dtype=complex)
    for a in range(2):
        for b in range(2):
            E0k[...,a,b]=fftn(E0[...,a,b])
    b=1j*np.einsum('...j,ijkl,...kl->...i',K,C,E0k)
    u_k=np.einsum('...ij,...j->...i',invA,b)
    u_k[mask0,...]=0.0
    Ux=u_k[...,0]; Uy=u_k[...,1]
    Exx=np.real(ifftn(1j*KX*Ux))
    Eyy=np.real(ifftn(1j*KY*Uy))
    Exy=np.real(ifftn(0.5j*(KX*Uy+KY*Ux)))
    DE=np.zeros_like(E0)
    DE[...,0,0]=Exx-E0[...,0,0]
    DE[...,1,1]=Eyy-E0[...,1,1]
    DE[...,0,1]=Exy; DE[...,1,0]=Exy
    f_el=0.5*np.einsum('...ij,ijkl,...kl->...',DE,C,DE).mean()
    fel.append(f_el)
    fc=f_chem(ci).mean() # uniform => scalar
    fchem.append(fc)
    ftotal.append(fc+f_el)

import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))
plt.plot(cs,np.array(fchem),label='f_chem')
plt.plot(cs,np.array(fel),label='f_el')
plt.plot(cs,np.array(ftotal),label='f_total')
plt.xlabel('c'); plt.ylabel('dimless energy density')
plt.legend(); plt.tight_layout()
plt.savefig('uniform_sweep_energy.png',dpi=150)
print('Saved: uniform_sweep_energy.png')

In [ ]:
#!/usr/bin/env python3
"""
Compute and plot the isotropically averaged structure factor S(k)
from a checkpoint (or any 2D c-field stored in NPZ).

Usage:
  python analyze_structure_factor.py [checkpoint.npz]
Saves: structure_factor.png and structure_factor.csv
"""
from __future__ import annotations
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fftn, fftshift, fftfreq
import sys

potential_fns = [arg for arg in sys.argv[1:] if arg.endswith('.npz')]
if potential_fns:
    fn = potential_fns[0]

data = np.load(fn, allow_pickle=True)
c=data['c']
Nx=int(data['Nx']); Ny=int(data['Ny'])
Lx=float(data['Lx']); Ly=float(data['Ly'])

ck=fftn(c)
Sk=np.abs(ck)**2
# frequencies (physical k)
kx=2*np.pi*fftfreq(Nx, d=Lx/Nx)
ky=2*np.pi*fftfreq(Ny, d=Ly/Ny)
KX,KY=np.meshgrid(kx,ky,indexing='ij')
K=np.sqrt(KX**2+KY**2)

# radial binning
kmax=float(np.max(K))
nbin=200
bins=np.linspace(0,kmax,nbin+1)
centers=0.5*(bins[:-1]+bins[1:])
Srad=np.zeros(nbin); counts=np.zeros(nbin)
inds=np.digitize(K.ravel(), bins)-1
for i in range(nbin):
    mask=(inds==i)
    if np.any(mask):
        Srad[i]=Sk.ravel()[mask].mean()
        counts[i]=mask.sum()

# characteristic length from peak
ipeak=np.argmax(Srad)
kstar=centers[ipeak]
length=2*np.pi/max(kstar,1e-12)
print(f"Peak k* = {kstar:.4f} (=> length ~ {length:.4f} in Wc units)")

# save csv
import pandas as pd
pd.DataFrame({'k':centers,'S(k)':Srad,'counts':counts}).to_csv('structure_factor.csv',index=False)

plt.figure(figsize=(5,3.5))
plt.plot(centers,Srad,'k-')
plt.xlabel('k [1/Wc]'); plt.ylabel('S(k)')
plt.title(f'Peak k*={kstar:.3f}, L~{length:.2f} Wc')
plt.tight_layout(); plt.savefig('structure_factor.png',dpi=150)
print('Saved: structure_factor.png, structure_factor.csv')

In [ ]:
# --- Save CSV log ---
import numpy as np
log = np.column_stack([times_s, currents_A, c_part_hist, c_dom_hist, V_hist])
np.savetxt(
    "iv_log.csv",
    log,
    delimiter=",",
    header="time_s,current_A,c_particle,c_domain,voltage_V",
    comments="",
)
print("Saved: iv_log.csv")

# --- Plots ---
import matplotlib.pyplot as plt

# (a) Current vs time
plt.figure(figsize=(6,3.8))
plt.plot(times_s, currents_A, 'k-')
plt.xlabel("time [s]")
plt.ylabel("current I [A]")
plt.title("Current vs Time (BV, smoothed boundary)")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("current_vs_time.png", dpi=150)
print("Saved: current_vs_time.png")

# (b) Voltage vs concentration (particle average)
plt.figure(figsize=(6,3.8))
plt.plot(c_part_hist, V_hist, 'b.-')
plt.xlabel("particle-averaged concentration ḉ_p")
plt.ylabel("cell potential V [V]")
plt.title("Potential vs Concentration")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("voltage_vs_concentration.png", dpi=150)
print("Saved: voltage_vs_concentration.png")


In [ ]:
#!/usr/bin/env python3
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt("iv_log.csv", delimiter=",", skiprows=1)
t, I, cpart, cdom, V = data.T

plt.figure(figsize=(6,3.8))
plt.plot(t, I, 'k-')
plt.xlabel("time [s]"); plt.ylabel("current I [A]")
plt.title("Current vs Time"); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig("current_vs_time.png", dpi=150)

plt.figure(figsize=(6,3.8))
plt.plot(cpart, V, 'b.-')
plt.xlabel("particle-averaged concentration ḉ_p"); plt.ylabel("potential V [V]")
plt.title("Potential vs Concentration"); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig("voltage_vs_concentration.png", dpi=150)

# Plot current vs concentration
plt.figure(figsize=(6,3.8))
plt.plot(cpart, I, 'r.-')
plt.xlabel("particle-averaged concentration ḉ_p")
plt.ylabel("current I [A]")
plt.title("Current vs Concentration")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("current_vs_concentration.png", dpi=150)


print("Saved: current_vs_time.png, voltage_vs_concentration.png, current_vs_concentration.png")

In [ ]:
#!/usr/bin/env python3
import os, glob
import imageio.v3 as iio

# Gather frames in numerical order
frames = sorted(glob.glob(os.path.join("frames", "frame_*.png")))
assert frames, "No frames found in ./frames!"

# MP4 (requires imageio-ffmpeg)
iio.imwrite(
    "concentration.mp4",
    [iio.imread(f) for f in frames],
    fps=10,                   # frames per second
    codec="libx264",          # H.264
    quality=8,                # 0 (best) ... 10 (worst) for imageio
    macro_block_size=1,       # avoid size constraints
)

# Optional GIF
iio.imwrite(
    "concentration.gif",
    [iio.imread(f) for f in frames],
    fps=10,
    loop=0
)

print("Wrote concentration.mp4 and concentration.gif")




In [ ]:
# (c) Particle-averaged concentration vs time
plt.figure(figsize=(6,3.8))
plt.plot(times_s, c_part_hist, 'g-')
plt.xlabel("time [s]")
plt.ylabel("particle-averaged concentration ḉ_p")
plt.title("Particle Concentration vs Time")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("particle_concentration_vs_time.png", dpi=150)
print("Saved: particle_concentration_vs_time.png")

In [ ]:
pwd


# 30 fps, 1080p-ish, quality-controlled MP4
ffmpeg -y -framerate 10 -pattern_type glob -i 'frames/frame_*.png' \
       -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" \
       -c:v libx264 -preset slow -crf 18 -pix_fmt yuv420p concentration.mp4



ffmpeg -y -framerate 10 -pattern_type glob -i 'frames/frame_*.png' \
       -vf "palettegen=stats_mode=diff" -y palette.png

ffmpeg -y -framerate 10 -pattern_type glob -i 'frames/frame_*.png' \
       -i palette.png -lavfi "paletteuse=dither=sierra2_4a" \
       concentration.gif